In [1]:
%load_ext autoreload
%autoreload 2

# Setup


In [3]:
import pandas as pd
from pathlib import Path
from modules.nlp import NLP

DATA_DIR = Path().cwd() / 'data'

posts_df = pd.read_csv(DATA_DIR / 'reddit_religion_combined.csv')

# Embeddings


**Pretrained embeddings:** Jeffrey Pennington, Richard Socher, and Christopher D. Manning. 2014. GloVe: Global Vectors for Word Representation. Download from [this link](https://nlp.stanford.edu/data/glove.840B.300d.zip).
